# Q1 scraping the  Wikipedia page

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
import requests
webseitdata=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(webseitdata,'html.parser')
#print(soup.prettify())

In [6]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [7]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text)

In [9]:
postalCodeList=list(map(lambda x:x.rstrip("\n"),postalCodeList))
boroughList=list(map(lambda x:x.rstrip("\n"),boroughList))
neighborhoodList=list(map(lambda x:x.rstrip("\n"),neighborhoodList))

In [13]:
df=pd.DataFrame({"PostalCode":postalCodeList, "Borough":boroughList,"Neighborhood":neighborhoodList})


In [12]:
# dropping the row 'not assigned'
df_clean1=df[df.Borough!="Not assigned"].reset_index(drop=True)
df_clean1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
# groupping neighborhoods in the same borough
toronto_df_grouped = df_clean1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
toronto_df_grouped["Neighborhood"]=toronto_df_grouped["Neighborhood"].replace("Not assigned",toronto_df_grouped["Borough"])
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
#checking
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

test_df=pd.DataFrame(columns=toronto_df_grouped.columns)

for postalcode in test_list:
    test_df=test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postalcode],ignore_index=True)
test_df
    

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [19]:
#print the rows
toronto_df_grouped.shape

(103, 3)

# Q2 creating dataframe with longitude and latitude value

In [20]:
geo_url="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(geo_url)

In [21]:
#geo_data.columns
geo_data.columns=['PostalCode', 'Latitude', 'Longitude']

In [23]:
toronto_df2=pd.merge(toronto_df_grouped,geo_data,how='inner',on="PostalCode")
toronto_df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#  Q3 Explore and cluster the neighborhoods in Toronto

In [24]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df2['Borough'].unique()),
        toronto_df2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [27]:
from geopy.geocoders import Nominatim 
import geopy
# convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [28]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto City are 43.6534817, -79.3839347.


In [32]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df2['Latitude'], toronto_df2['Longitude'], toronto_df2['Borough'], toronto_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

In [33]:
## Step 3c. Foursquare to explore
df_borough_toronto=toronto_df2[toronto_df2["Borough"].str.contains("Toronto")].reset_index(drop=True)
df_borough_toronto.size
df_borough_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [34]:
df_borough_toronto["Borough"].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [35]:
df_borough_toronto["color"]=df_borough_toronto["Borough"].map({'East Toronto':"green", 'Central Toronto':"red", 'Downtown Toronto':"blue",
       'West Toronto':"black"})

In [36]:
df_borough_toronto.shape

(39, 6)

In [40]:
CLIENT_ID = '0GPNYOVQQJBQDIIH3TCFRMKBQKP5ZNKBH5CB4LI3VMD51442' # your Foursquare ID
CLIENT_SECRET = '33LVENI2BSLMN0OWWDSYCSMMAOWLZHAUXA1D4KTXYPI0A2HR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0GPNYOVQQJBQDIIH3TCFRMKBQKP5ZNKBH5CB4LI3VMD51442
CLIENT_SECRET:33LVENI2BSLMN0OWWDSYCSMMAOWLZHAUXA1D4KTXYPI0A2HR


In [41]:
#first neigbourhood
neighborhood_latitude1=df_borough_toronto["Latitude"][0]
neighborhood_longitude1=df_borough_toronto["Longitude"][0]
neighborhood_name1=df_borough_toronto["Neighborhood"][0]

print (f"{neighborhood_name1} has lognitude and latitude as : [{neighborhood_latitude1},{neighborhood_longitude1}]")

The Beaches has lognitude and latitude as : [43.67635739999999,-79.2930312]


In [42]:
# Setup API URL to explore venues near by
LIMIT=100
RADIUS=500
url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={neighborhood_latitude1},{neighborhood_longitude1}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]

In [43]:
# Serializing json
import json
json_object = json.dumps(neighborhood_json, indent = 4)

In [44]:
#save data as json file to explore
with open("jsonData.json","w") as f:
    f.write(json_object)

In [45]:
venues=neighborhood_json

In [46]:
#flatten Json
from pandas.io.json import json_normalize
nearby_venues=json_normalize(venues)

In [47]:
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

In [48]:
nearby_venues=nearby_venues.loc[:,filtered_columns]

In [49]:
def getCategory_type(row):
    try:
        category_list=row["name"]
    except:
        category_list=row["venue.categories"]
    if len(category_list)==0:
        return None
    else:
        return category_list[0]["name"]

In [50]:
nearby_venues["categories"]= [x[0]["name"] for x in nearby_venues["venue.categories"]]

In [51]:
nearby_venues.drop(["venue.categories"],axis=1,inplace=True)

In [52]:
nearby_venues

,venue.name,venue.location.lat,venue.location.lng,categories
0,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Upper Beaches,43.680563,-79.292869,Neighborhood


In [53]:
#we have just explored one neighbourhood and its nearby venues and their category now we will be doing for all nerwighborhood in
df_borough_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude,color
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,green
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,green
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,green
3,M4M,East Toronto,Studio District,43.659526,-79.340923,green
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,red
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,red
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,red
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,red
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,red
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,red


In [54]:
CLIENT_ID = 'X5STHTOJ5ALK42QMTYDXRXWTKQ20FB4OOWKA5IQZ4PE1D5ZI' # your Foursquare ID
CLIENT_SECRET = 'VPEBQYOI35SAJ5EOP5L34DXJTRQIKHNT5ERK051GFSSQKNWU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
# Setup API URL to explore venues near by
LIMIT=100
RADIUS=500

def getNearByVenues(neighbourhood_name,lat,long):
    venues_list=[]

    for name, lat, lng in zip(neighbourhood_name,lat,long):
        print(name)
        
        url=f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{lng}&v={VERSION}&radius={RADIUS}&limit={LIMIT}"
        neighborhood_json = requests.get(url).json()["response"]["groups"][0]["items"]
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in neighborhood_json])
        #appending list of  venuedetails as list into another list venues list
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood', 
                    'Neighborhood Latitude', 
                    'Neighborhood Longitude', 
                    'Venue', 
                    'Venue Latitude', 
                    'Venue Longitude', 
                    'Venue Category']
    return (nearby_venues)

In [55]:
#Get all Tor
toronto_venues_df = getNearByVenues(df_borough_toronto['Neighborhood'],df_borough_toronto['Latitude'],df_borough_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [56]:
toronto_venues_df.shape
#toronto_venues_df.head()

(1634, 7)

In [57]:
toronto_venues_df.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,66,66,66,66,66,66
Christie,16,16,16,16,16,16
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [58]:
#analyze the neighbourhoood
#creating dummy for each venue category

torento_onehot=pd.get_dummies(toronto_venues_df[["Venue Category"]], prefix="", prefix_sep="")

In [59]:
torento_onehot.shape

(1634, 234)

In [60]:
torento_onehot["Neighborhood"]=toronto_venues_df["Neighborhood"]

In [61]:
torento_onehot.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [62]:
torento_onehot.columns.get_loc("Neighborhood")

162

In [100]:
torento_onehot.columns[162]

'Neighborhood'

In [103]:
fixed_columns=[torento_onehot.columns[162]]+list(torento_onehot.columns[0:162])+list(torento_onehot.columns[163:])

In [104]:
len(fixed_columns)

234

In [105]:
len(fixed_columns)

234

In [106]:
torento_onehot.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [107]:
torento_onehot_grouped=torento_onehot.groupby("Neighborhood").mean().reset_index()

In [108]:
torento_onehot_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.0000,0.017544,0.0000,0.017544,0.035088,0.000000,0.000000,0.00,0.017544,0.017544,0.000000,0.035088,0.000000,0.000000,0.017544,0.0000,0.000000,0.000000,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.035088,0.00,0.0000,0.000000,0.035088,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.070175,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.017544,0.000000,0.017544,0.000000,0.00,0.000000,0.00000,0.017544,0.000000,0.000000,0.000000,0.000000,0.017544,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,

In [109]:
for  hood in torento_onehot_grouped["Neighborhood"]:
    print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    print(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10))
    dict1={}
    print("\n")

-------Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3      Farmers Market  0.04
4            Beer Bar  0.04
5            Pharmacy  0.04
6          Restaurant  0.04
7                Café  0.04
8         Cheese Shop  0.04
9              Bakery  0.04


-------Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3       Convenience Store  0.05
4      Italian Restaurant  0.05
5                 Stadium  0.05
6  Furniture / Home Store  0.05
7            Climbing Gym  0.05
8               Nightclub  0.05
9                  Bakery  0.05


-------Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1               Brewery  0.06
2         Garden Center  0.06
3                  Park  0.06
4          

In [110]:
dict1={}

for  hood in torento_onehot_grouped["Neighborhood"]:
    val=[]
    #print(f"-------{hood}----")
    temp=torento_onehot_grouped[torento_onehot_grouped["Neighborhood"]==hood].T.reset_index()
    temp.columns=["venue","freq"]
    temp=temp[1:]
    temp["freq"]=round(temp["freq"].astype(float),2)
    val=list(temp.sort_values(by="freq",axis=0,ascending=False).reset_index(drop=True).head(10)["venue"])
    dict1[hood]=val

In [111]:
cols=["No."+str(x)+"_common_Place" for x in range(1,11)]

In [119]:
neighborhoods_venues_sorted=pd.DataFrame(dict1).T

In [120]:
neighborhoods_venues_sorted.columns=cols

In [121]:
neighborhoods_venues_sorted.insert(0,"Neighborhood",list(neighborhoods_venues_sorted.index))

In [122]:
neighborhoods_venues_sorted.reset_index(drop=True,inplace=True)

In [123]:
neighborhoods_venues_sorted.head()

,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Pharmacy,Restaurant,Café,Cheese Shop,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Italian Restaurant,Stadium,Furniture / Home Store,Climbing Gym,Nightclub,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Garden Center,Park,Comic Shop,Recording Studio,Restaurant,Farmers Market,Skate Park,Fast Food Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Rental Car Location,Plane,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Thai Restaurant,Burger Joint,Bubble Tea Shop,Salad Place,Department Store


In [124]:
torento_onehot_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.0000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0000,0.0,0.0000,0.0,0.017544,0.0000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.000000,0.0,0.052632,0.070175,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.0000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0000,0.0,0.000000,0.0,0

In [125]:
#Training
from  sklearn.cluster import KMeans

#set no of clusters
n_cluster=5
#set gtraining Data
training_Data=torento_onehot_grouped.drop("Neighborhood",axis=1)
#Training the model
cluster_kmean=KMeans(n_clusters=n_cluster,random_state=0).fit(training_Data)
cluster_kmean

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [126]:
#check the labels
cluster_kmean.labels_

array([0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 3, 0, 1, 0, 0, 3, 3, 3, 1, 0, 4, 0,
       3, 0, 0, 0, 1, 2, 3, 0, 3, 0, 3, 3, 3, 3, 3, 0, 3], dtype=int32)

In [127]:
#adding cluster into venues tables
neighborhoods_venues_sorted.insert(0,"cluster_lablel",cluster_kmean.labels_)

In [128]:
neighborhoods_venues_sorted

,cluster_lablel,Neighborhood,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
0,0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Pharmacy,Restaurant,Café,Cheese Shop,Bakery
1,3,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Italian Restaurant,Stadium,Furniture / Home Store,Climbing Gym,Nightclub,Bakery
2,3,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Garden Center,Park,Comic Shop,Recording Studio,Restaurant,Farmers Market,Skate Park,Fast Food Restaurant
3,0,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Rental Car Location,Plane,Boutique,Boat or Ferry,Sculpture Garden,Coffee Shop
4,0,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Thai Restaurant,Burger Joint,Bubble Tea Shop,Salad Place,Department Store
5,3,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Diner,Baby Store,Italian Restaurant,Restaurant
6,0,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Pub,Yoga Studio
7,0,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
8,3,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Gym,Sushi Restaurant,Café,Italian Restaurant,Farmers Market,Brewery
9,3,Davisville North,Gym / Fitness Center,Dance Studio,Pizza Place,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Hotel,Park,Organic Grocery


In [129]:
torento_merged=toronto_df2.copy()

In [130]:
torento_merged=pd.merge(torento_merged,neighborhoods_venues_sorted,on="Neighborhood")

In [131]:
torento_merged.set_index("PostalCode",drop=True,inplace=True)

In [132]:
torento_merged

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Trail,Airport,Music Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Lounge,Bookstore,Fruit & Vegetable Store
M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Pizza Place,Sandwich Place,Brewery,Sushi Restaurant,Pub,Coffee Shop,Restaurant,Movie Theater,Fast Food Restaurant
M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Brewery,Gay Bar,Cheese Shop,Clothing Store,Comfort Food Restaurant
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Airport,Movie Theater,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Gym / Fitness Center,Dance Studio,Pizza Place,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Hotel,Park,Organic Grocery
M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Park,Diner,Rental Car Location,Restaurant,Salon / Barbershop,Chinese Restaurant
M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Gym,Sushi Restaurant,Café,Italian Restaurant,Farmers Market,Brewery
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Playground,Summer Camp,Tennis Court,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


In [133]:
torento_merged.columns

Index(['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'cluster_lablel',
       'No.1_common_Place', 'No.2_common_Place', 'No.3_common_Place',
       'No.4_common_Place', 'No.5_common_Place', 'No.6_common_Place',
       'No.7_common_Place', 'No.8_common_Place', 'No.9_common_Place',
       'No.10_common_Place'],
      dtype='object')

In [134]:
# create map of Torronto using latitude and longitude values
map_toronto = folium.Map(location=[torento_merged["Latitude"][0], torento_merged["Longitude"][0]], zoom_start=10)

# set color scheme for the clusters
x = np.arange(n_cluster)
ys = [i + x + (i*x)**2 for i in range(n_cluster)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng,neighborhood,cluster_label in zip(torento_merged['Latitude'], torento_merged['Longitude'], torento_merged['Neighborhood'],torento_merged["cluster_lablel"]):
    
    label = folium.Popup(str(neighborhood)+"cluster\n"+str(cluster_label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster_label],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

In [135]:
# exam clusters cluster1
torento_merged[torento_merged["cluster_lablel"]== 0]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Park,Diner,Rental Car Location,Restaurant,Salon / Barbershop,Chinese Restaurant
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Pub,Yoga Studio
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Theater,Pub,Breakfast Spot,Farmers Market,Chocolate Shop,Cosmetics Shop
M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Hotel,Bubble Tea Shop,Fast Food Restaurant,Bookstore
M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Clothing Store,American Restaurant,Cosmetics Shop,Cocktail Bar,Lingerie Store,Breakfast Spot,Gym
M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Pharmacy,Restaurant,Café,Cheese Shop,Bakery
M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sandwich Place,Café,Thai Restaurant,Burger Joint,Bubble Tea Shop,Salad Place,Department Store
M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,American Restaurant,Steakhouse
M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,0,Coffee Shop,Aquarium,Café,Hotel,Brewery,Restaurant,Scenic Lookout,Sporting Goods Shop,Fried Chicken Joint,Bakery


In [136]:
# cluster 2
torento_merged[torento_merged["cluster_lablel"]== 1]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Airport,Movie Theater,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Park,Trail,Sushi Restaurant,Jewelry Store,Airport,Movie Theater,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


In [137]:
# cluster 3
torento_merged[torento_merged["cluster_lablel"]== 2]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden,Airport,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


In [138]:
# cluster 4
torento_merged[torento_merged["cluster_lablel"]== 3]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Pub,Trail,Airport,Music Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Restaurant,Bubble Tea Shop,Lounge,Bookstore,Fruit & Vegetable Store
M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Pizza Place,Sandwich Place,Brewery,Sushi Restaurant,Pub,Coffee Shop,Restaurant,Movie Theater,Fast Food Restaurant
M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,American Restaurant,Gastropub,Bakery,Brewery,Gay Bar,Cheese Shop,Clothing Store,Comfort Food Restaurant
M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Gym / Fitness Center,Dance Studio,Pizza Place,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Hotel,Park,Organic Grocery
M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Gym,Sushi Restaurant,Café,Italian Restaurant,Farmers Market,Brewery
M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,3,Pub,Coffee Shop,Sports Bar,Supermarket,Bank,Restaurant,Fried Chicken Joint,Light Rail Station,Sushi Restaurant,Sandwich Place
M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,3,Coffee Shop,Pub,Market,Café,Restaurant,Pizza Place,Bakery,Pet Store,Italian Restaurant,General Entertainment
M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Café,Sandwich Place,Coffee Shop,Pharmacy,BBQ Joint,Pizza Place,Liquor Store,Donut Shop,Burger Joint,Middle Eastern Restaurant


In [139]:
# cluster 5
torento_merged[torento_merged["cluster_lablel"]== 4]

,Borough,Neighborhood,Latitude,Longitude,cluster_lablel,No.1_common_Place,No.2_common_Place,No.3_common_Place,No.4_common_Place,No.5_common_Place,No.6_common_Place,No.7_common_Place,No.8_common_Place,No.9_common_Place,No.10_common_Place
PostalCode,,,,,,,,,,,,,,,
M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.38316,4,Playground,Summer Camp,Tennis Court,Airport,Museum,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


Observations: Most of the neighborhoods fall into Cluster 4 which are mostly business areas with cafe, restaurants etc. 